In [ ]:

import sys
from pathlib import Path
import seaborn as sns

In [ ]:

LCL_PATH  = str(Path().cwd().parent.parent)
ROOT_PATH = str(Path(LCL_PATH).parent.parent)
EXTERN_CMD_PATH = str(Path(ROOT_PATH))
GENERIC_PATH    = str(Path(ROOT_PATH)/"GeneticAlgorithmManager")
SPECIFIC_PATH   = str(Path(ROOT_PATH)/"extension")
print("""
root path:\t{}
local path:\t{}
generic_path:\t{}
project_specific_path:\t{}
extern_cmd_path: \t{}""".format(ROOT_PATH, LCL_PATH, GENERIC_PATH, SPECIFIC_PATH, EXTERN_CMD_PATH))

In [ ]:

# adding local_folder to the system path
sys.path.append(LCL_PATH)
sys.path.append(ROOT_PATH)
sys.path.append(GENERIC_PATH)
sys.path.append(SPECIFIC_PATH)

from sys_function import * # este in root

In [ ]:

sys_remove_modules("imshow")
sys_remove_modules("algoritm_genetic") # facem update doar daca modificam ceva in generic
sys_remove_modules("genoms") # 
sys_remove_modules("builds.init_population.init_population_tabu_search_ttp")
sys_remove_modules("builds.dataset_utils.dataset_ttp_man")
sys_remove_modules("builds.dataset_utils.ttp_generator")

from imshow import *
from algoritm_genetic import *
from genoms import *
from builds.init_population.init_population_tabu_search_ttp import *
from builds.dataset_utils.dataset_ttp_man import *
from builds.dataset_utils.ttp_generator import *

# Data aquisition

In [ ]:

ttp_generator = TTPGenerator("{}/data".format(LCL_PATH))
dataset = ttp_generator(nodes_file="NODE_COORD_SECTION.csv",
                        items_file="ITEMS_SECTION.csv")
dataset

In [ ]:
dataset_obj = DatasetTTPMan(dataset)

# InitPopulationTabuSearch

In [ ]:

obj = InitPopulationTabuSearch("init", dataset_man=dataset_obj, city=0, window_size=4, v_min=0.1, v_max=1, W=25936, R=5.61)
obj.setParameters(GENOME_LENGTH=dataset["GENOME_LENGTH"])

## __call__

In [ ]:

genoms = obj(1)

In [ ]:

for elite_pos in np.arange(2):
    print(elite_pos)
    tsp_individ = genoms["tsp"][elite_pos]
    kp_individ  = genoms["kp"][elite_pos]
    score = dataset_obj.computeIndividScore(tsp_individ, kp_individ, v_min=0.1, v_max=1, W=25936, R=5.61)
    profit = dataset_obj.computeIndividProfit(kp_individ)
    weight = dataset_obj.computeIndividWeight(kp_individ)
    print("score {}, weight {}, profit {}".format(score, weight, profit))
    map_imgp = ttp_generator.putRoutesOnMap(tsp_individ)
    image_show_wait(map_imgp, window_name="Parent")

## computeRoute

## computeProfit

In [ ]:

ret_route = np.arange(obj.GENOME_LENGTH, dtype=np.int32)
for idx in range(2):
    tmp_profit = obj.computeProfit(1, [ret_route], v_min=0.1, v_max=1, W=25936, R=5.61)
    score  = dataset_obj.computeIndividScore(ret_route, tmp_profit[0], v_min=0.1, v_max=1, W=25936, R=5.61)
    profit = dataset_obj.computeIndividProfit(tmp_profit)
    weight = dataset_obj.computeIndividWeight(tmp_profit)
    print("score {}, weight {}, profit {}".format( score, weight, profit))

In [ ]:

score  = dataset_obj.computeIndividScore(ret_route, tmp_profit[0], v_min=0.1, v_max=1, W=25936, R=5.61)
profit = dataset_obj.computeIndividProfit(tmp_profit[0])
weight = dataset_obj.computeIndividWeight(tmp_profit[0])
print("score {}, weight {}, profit {}".format( score, weight, profit))

## insertion_tabu_search_distance

In [ ]:

route, city = np.arange(280, dtype=np.int32), 0
is_find = True
count = 0
ret_route = route.copy()
while (is_find and (count < 1000)):
    ret_route, is_find = obj.insertion_tabu_search_distance(ret_route, city)
    old_distance = dataset_obj.computeIndividDistance(route)
    new_distance = dataset_obj.computeIndividDistance(ret_route)
    count += 1
print("old distance {}, new distance {}, count {}".format(old_distance, new_distance, count))
map_imgo = ttp_generator.putRoutesOnMap(route)
map_imgp = ttp_generator.putRoutesOnMap(ret_route)
image_show(map_imgo, window_name="Old")
image_show_wait(map_imgp, window_name="Tabu")

## tabu_search_score

In [ ]:

tmp_profit = np.random.randint(low=0, high=2, size=obj.GENOME_LENGTH)
for idx in range(10):
    tmp_profit, is_find = obj.tabu_search_score(ret_route, tmp_profit, v_min=0.1, v_max=1, W=25936, R=5.61)
    tmp_profit = obj.erase_weightier_objects(tmp_profit, 25936)
    score  = dataset_obj.computeIndividScore(ret_route, tmp_profit, v_min=0.1, v_max=1, W=25936, R=5.61)
    profit = dataset_obj.computeIndividProfit(tmp_profit)
    weight = dataset_obj.computeIndividWeight(tmp_profit)
    print("is_find {}, score {}, weight {}, profit {}".format(is_find, score, weight, profit))
    